<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-18 09:52:45
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.33 C
CY Investment:  1.60 C
Reserve:  4.98 L
Current:  1.43 C
-------------------
Today PnL: -16.25 K (-0.11%)
Current PnL: -31.48 L (-19.67%)
CY Booked + Current PnL: -15.80 L (-9.87%)
-------------------
Total profit:  1.35 L
Total loss:  -32.82 L
-------------------
Total Booked + Current PnL: 11.18 L (8.41%)
Total Booked PnL: 42.66 L (32.07%)
Curr Year Booked PnL: 15.68 L (10.97%)
Prev Year Booked PnL: 26.98 L (20.29%)
Est FTT:  2.40 C
Est FTT PnL: 96.82 L (67.72%)
Deployed:  1.33 C
Current:  1.43 C
CAGR/XIRR %: 3.75%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,0.06,3.49,17.47,21.57,17715.0,3420.0,101400.0,397.04,-10.57,60.0,H-SC,4.90,167.0,0.19,0.73,47.97,XR,ATH,FINANCE
18,COALINDIA,-0.06,2.25,15.08,17.67,23200.0,3386.0,153846.0,478.51,23.39,49.0,L-LC,11.34,185.0,0.15,1.11,23.50,XY25,ATH,MINING
77,TTKPRESTIG,-0.06,-27.36,37.78,0.09,27658.0,-27568.0,73209.0,770.00,69.90,39.0,M-SC,9.35,250.0,-1.00,0.53,0.76,OX40N,NTT,DURABLES
36,ICICIGI,1.46,3.63,15.16,19.34,31437.0,7259.0,207368.0,2252.93,-12.72,65.0,X-MC,2.83,66.0,0.23,1.50,20.14,X40,ATH,INSURANCE
51,MEDANTA,-0.63,-2.73,27.55,24.08,34021.0,-3460.0,123490.0,1486.00,-3.46,56.0,X-SC,11.05,87.0,-0.10,0.89,13.11,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,EASEMYTRIP,2.00,-46.52,172.45,45.70,163060.0,-82260.0,94555.0,26.40,-1.77,82.0,M-SC,22.93,204.0,-0.50,0.69,55.79,XY24,NTT,TRAVEL
11,BANDHANBNK,1.60,-17.68,133.75,92.42,306690.0,-49258.0,229301.0,400.00,96.84,71.0,H-SC,7.04,174.0,-0.16,1.66,32.52,XY24,NTT,BANKS
36,ICICIGI,1.46,3.63,15.16,19.34,31437.0,7259.0,207368.0,2252.93,-12.72,65.0,X-MC,2.83,66.0,0.23,1.50,20.14,X40,ATH,INSURANCE
80,VALIANTORG,1.33,-49.10,242.15,74.15,201646.0,-80332.0,83273.0,838.00,-249.72,42.0,H-SC,24.11,145.0,-0.40,0.60,6.74,XR,NTT,CHEMICALS
44,ITC,1.27,-19.22,37.14,10.78,72101.0,-46184.0,194134.0,452.00,-52.28,53.0,X-LC,18.69,1.0,-0.64,1.41,6.93,X40,NTT,FMCG


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,DIXON,-2.41,-21.26,66.71,31.26,121209.0,-49072.0,181696.0,18931.72,-64.81,48.0,X-MC,25.71,54.0,-0.40,1.32,11.51,X40N,ATH,IT
42,INFY,-2.34,-7.51,45.14,34.24,130637.0,-23485.0,289403.0,1972.00,-27.07,23.0,X-LC,3.93,3.0,-0.18,2.10,0.15,X40,NTT,IT
14,BSOFT,-1.83,-31.32,119.56,50.79,115488.0,-44059.0,96594.0,831.70,-11.87,37.0,H-SC,7.02,172.0,-0.38,0.70,12.47,XR,ATH,IT
84,WIPRO,-1.82,-13.15,98.34,72.27,144730.0,-22275.0,147173.0,420.00,-21.74,27.0,M-LC,6.44,98.0,-0.15,1.07,0.00,XR,NTT,IT
21,DABUR,-1.68,-0.66,44.66,43.70,110055.0,-1640.0,246428.0,735.00,-9.86,46.0,X-MC,1.87,70.0,-0.01,1.79,14.29,XY24,BTT,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,0.33,4.04,99.85,107.91,153372.0,5958.0,153602.0,1641.55,-6.65,66.0,M-SC,8.78,220.0,0.04,1.11,15.56,OX40N,ATH,CABLES
77,TTKPRESTIG,-0.06,-27.36,37.78,0.09,27658.0,-27568.0,73209.0,770.00,69.90,39.0,M-SC,9.35,250.0,-1.00,0.53,0.76,OX40N,NTT,DURABLES
66,SIS,0.45,-28.54,69.69,21.26,55294.0,-31689.0,79343.0,528.00,1866.32,37.0,H-SC,4.23,168.0,-0.57,0.58,9.38,OX40N,NTT,MISC
72,TATAELXSI,-1.49,-30.17,89.06,32.03,81992.0,-39771.0,92064.0,9161.00,-12.63,35.0,H-SC,7.26,158.0,-0.49,0.67,3.12,OX40N,NTT,IT
40,INDIGOPNTS,-0.61,-30.95,44.86,0.02,54069.0,-54031.0,120528.0,1408.00,83.59,30.0,M-SC,8.69,222.0,-1.00,0.87,4.84,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,0.53,20.3,20.69,45.2,47726.0,38928.0,230670.0,1856.0,37.67,67.0,X-MC,4.49,79.0,0.82,1.67,28.48,XY25,NTT,AC


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-0.06,2.25,15.08,17.67,23200.0,3386.0,153846.0,478.51,23.39,49.0,L-LC,11.34,185.0,0.15,1.11,23.50,XY25,ATH,MINING
26,FINCABLES,0.33,4.04,99.85,107.91,153372.0,5958.0,153602.0,1641.55,-6.65,66.0,M-SC,8.78,220.0,0.04,1.11,15.56,OX40N,ATH,CABLES
50,MASFIN,0.06,3.49,17.47,21.57,17715.0,3420.0,101400.0,397.04,-10.57,60.0,H-SC,4.90,167.0,0.19,0.73,47.97,XR,ATH,FINANCE
1,ABB,-0.05,7.02,36.02,45.57,100850.0,18365.0,279984.0,7934.00,-31.20,68.0,H-MC,4.30,125.0,0.18,2.03,24.33,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,0.06,3.49,17.47,21.57,17715.0,3420.0,101400.0,397.04,-10.57,60.0,H-SC,4.90,167.0,0.19,0.73,47.97,XR,ATH,FINANCE
1,ABB,-0.05,7.02,36.02,45.57,100850.0,18365.0,279984.0,7934.00,-31.20,68.0,H-MC,4.30,125.0,0.18,2.03,24.33,AR,NTT,ELECTRICAL
26,FINCABLES,0.33,4.04,99.85,107.91,153372.0,5958.0,153602.0,1641.55,-6.65,66.0,M-SC,8.78,220.0,0.04,1.11,15.56,OX40N,ATH,CABLES
39,INDIAMART,-1.03,-5.43,118.59,106.72,138320.0,-6699.0,116637.0,4810.62,-57.42,47.0,H-SC,7.79,140.0,-0.05,0.85,16.68,AR,ATH,MISC
85,ZYDUSLIFE,-0.07,-4.46,41.96,35.63,84015.0,-9348.0,200226.0,1286.17,-16.67,51.0,H-MC,5.43,120.0,-0.11,1.45,11.47,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,-2.34,-7.51,45.14,34.24,130637.0,-23485.0,289403.0,1972.00,-27.07,23.0,X-LC,3.93,3.0,-0.18,2.10,0.15,X40,NTT,IT
73,TCS,-1.08,-26.09,60.40,18.55,168855.0,-98675.0,279562.0,4311.59,-33.14,26.0,X-LC,4.22,2.0,-0.58,2.03,0.00,X40,ATH,IT
33,HCLTECH,-1.16,-4.90,30.22,23.83,69527.0,-11856.0,230068.0,1908.19,0.08,27.0,X-LC,4.54,8.0,-0.17,1.67,10.38,X40,ATH,IT
8,AWL,-0.48,-34.18,138.82,57.19,297985.0,-111471.0,214656.0,485.00,-62.33,30.0,X-SC,17.55,81.0,-0.37,1.56,0.75,XY24,NTT,FMCG
13,BERGEPAINT,-0.31,-17.70,47.20,21.14,88306.0,-40249.0,187090.0,680.00,-19.50,30.0,X-MC,12.41,68.0,-0.46,1.36,0.75,XY24,NTT,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,-0.12,4.48,15.59,20.77,38139.0,10492.0,244636.0,1644.00,-7.40,63.0,X-MC,1.47,64.0,0.28,1.77,12.21,X40N,NTT,BREWERIES
21,DABUR,-1.68,-0.66,44.66,43.70,110055.0,-1640.0,246428.0,735.00,-9.86,46.0,X-MC,1.87,70.0,-0.01,1.79,14.29,XY24,BTT,FMCG
65,SIEMENS,-0.22,-14.57,46.93,25.51,74607.0,-27120.0,158975.0,4671.50,33.57,56.0,H-LC,1.96,51.0,-0.36,1.15,17.15,AR,ATH,ELECTRICAL
36,ICICIGI,1.46,3.63,15.16,19.34,31437.0,7259.0,207368.0,2252.93,-12.72,65.0,X-MC,2.83,66.0,0.23,1.50,20.14,X40,ATH,INSURANCE
34,HINDUNILVR,0.09,-9.97,26.25,13.67,60753.0,-25623.0,231440.0,2922.00,-33.13,41.0,X-LC,2.87,9.0,-0.42,1.68,9.26,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PGHH,-0.28,-13.38,52.40,32.01,96919.0,-28560.0,184960.0,17616.87,-41.36,34.0,X-MC,9.33,57.0,-0.29,1.34,0.00,X40,ATH,FMCG
73,TCS,-1.08,-26.09,60.40,18.55,168855.0,-98675.0,279562.0,4311.59,-33.14,26.0,X-LC,4.22,2.0,-0.58,2.03,0.00,X40,ATH,IT
42,INFY,-2.34,-7.51,45.14,34.24,130637.0,-23485.0,289403.0,1972.00,-27.07,23.0,X-LC,3.93,3.0,-0.18,2.10,0.15,X40,NTT,IT
3,ACC,-0.27,-31.29,138.90,64.15,227102.0,-74451.0,163500.0,3906.00,-48.05,38.0,X-SC,8.49,85.0,-0.33,1.18,0.17,XY24,BTT,CEMENT
13,BERGEPAINT,-0.31,-17.70,47.20,21.14,88306.0,-40249.0,187090.0,680.00,-19.50,30.0,X-MC,12.41,68.0,-0.46,1.36,0.75,XY24,NTT,PAINTS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,-0.09,-30.12,104.41,42.84,47430.0,-19579.0,45427.0,424.00,-54.27,51.0,X-SC,18.85,80.0,-0.41,0.33,9.27,XY24,NTT,MISC
58,RELAXO,0.49,-52.09,217.88,52.31,151538.0,-75609.0,69551.0,1176.00,-48.99,46.0,X-SC,14.06,92.0,-0.50,0.50,4.27,X40N,NTT,FOOTWEAR
12,BATAINDIA,0.11,-45.83,149.60,35.20,104270.0,-58971.0,69699.0,2096.00,-8.59,38.0,X-SC,21.50,91.0,-0.57,0.51,1.32,X40,NTT,FOOTWEAR
51,MEDANTA,-0.63,-2.73,27.55,24.08,34021.0,-3460.0,123490.0,1486.00,-3.46,56.0,X-SC,11.05,87.0,-0.10,0.89,13.11,XY24,NTT,HEALTHCARE
35,HONAUT,0.10,-22.87,85.82,43.32,107807.0,-37252.0,125620.0,58357.33,-32.25,39.0,X-SC,11.15,90.0,-0.35,0.91,2.00,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,1.27,-19.22,37.14,10.78,72101.0,-46184.0,194134.0,452.00,-52.28,53.0,X-LC,18.69,1.0,-0.64,1.41,6.93,X40,NTT,FMCG
73,TCS,-1.08,-26.09,60.40,18.55,168855.0,-98675.0,279562.0,4311.59,-33.14,26.0,X-LC,4.22,2.0,-0.58,2.03,0.00,X40,ATH,IT
42,INFY,-2.34,-7.51,45.14,34.24,130637.0,-23485.0,289403.0,1972.00,-27.07,23.0,X-LC,3.93,3.0,-0.18,2.10,0.15,X40,NTT,IT
75,TMPV,0.12,-21.44,56.54,22.98,100340.0,-48430.0,177468.0,600.00,-81.44,64.0,X-LC,2.94,4.0,-0.48,1.29,13.45,XY24,NTT,AUTO
33,HCLTECH,-1.16,-4.90,30.22,23.83,69527.0,-11856.0,230068.0,1908.19,0.08,27.0,X-LC,4.54,8.0,-0.17,1.67,10.38,X40,ATH,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,-0.85,-30.91,92.35,32.89,49260.0,-23866.0,53341.0,1800.00,-351.40,60.0,L-MC,11.08,258.0,-0.48,0.39,46.90,XR,NTT,BANKS
67,SONACOMS,-0.63,-8.94,52.74,39.09,48584.0,-9039.0,92120.0,804.02,-27.90,61.0,M-MC,6.65,193.0,-0.19,0.67,30.32,AR,ATH,AUTO
25,EASEMYTRIP,2.00,-46.52,172.45,45.70,163060.0,-82260.0,94555.0,26.40,-1.77,82.0,M-SC,22.93,204.0,-0.50,0.69,55.79,XY24,NTT,TRAVEL
50,MASFIN,0.06,3.49,17.47,21.57,17715.0,3420.0,101400.0,397.04,-10.57,60.0,H-SC,4.90,167.0,0.19,0.73,47.97,XR,ATH,FINANCE
79,VAIBHAVGBL,-1.49,-23.63,110.51,60.77,154254.0,-43191.0,139584.0,521.00,60.88,56.0,H-SC,1.47,160.0,-0.28,1.01,26.55,XR,NTT,JEWELLERY


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,EASEMYTRIP,2.00,-46.52,172.45,45.70,163060.0,-82260.0,94555.0,26.40,-1.77,82.0,M-SC,22.93,204.0,-0.50,0.69,55.79,XY24,NTT,TRAVEL
7,ATULAUTO,0.11,-11.16,63.85,45.57,111823.0,-21998.0,175134.0,844.00,3797.30,69.0,M-SC,4.41,248.0,-0.20,1.27,32.54,XY24,NTT,AUTO
1,ABB,-0.05,7.02,36.02,45.57,100850.0,18365.0,279984.0,7934.00,-31.20,68.0,H-MC,4.30,125.0,0.18,2.03,24.33,AR,NTT,ELECTRICAL
11,BANDHANBNK,1.60,-17.68,133.75,92.42,306690.0,-49258.0,229301.0,400.00,96.84,71.0,H-SC,7.04,174.0,-0.16,1.66,32.52,XY24,NTT,BANKS
67,SONACOMS,-0.63,-8.94,52.74,39.09,48584.0,-9039.0,92120.0,804.02,-27.90,61.0,M-MC,6.65,193.0,-0.19,0.67,30.32,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.20
1,25,44.67
2,50,76.76


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    46.43
MC    30.35
LC    23.24
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.94
X40      23.20
X40N     15.18
AR        9.00
XY25      8.80
XR        8.71
OX40N     7.31
SR        0.88
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    22.91
H-SC    22.73
X-LC    19.91
X-SC    11.85
M-SC    11.06
H-MC     4.94
M-MC     2.11
H-LC     1.15
L-LC     1.11
M-LC     1.07
L-SC     0.79
L-MC     0.39
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.04,-14.16,52.53
FINANCE,13.43,-12.61,56.93
IT,11.45,-35.61,103.28
MISC,7.09,-32.97,87.36
ELECTRICAL,6.27,-8.45,48.02
PAINTS,5.09,-31.53,51.07
INSURANCE,4.91,-0.09,34.33
PHARMA,4.13,-3.08,35.52
AUTO,3.23,-17.87,58.63


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3257151.0,21
AR,1330832.0,10
X40,1296361.0,15
XR,1286988.0,12
X40N,1025048.0,10
OX40N,784619.0,10
XY25,402564.0,6
SR,298080.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3658668.0,24
M-SC,1392215.0,14
X-MC,1353258.0,15
X-SC,1167212.0,10
X-LC,1145401.0,12
H-MC,396871.0,3
L-SC,181332.0,2
M-LC,144730.0,1
M-MC,94889.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1235757.0      6
           AR         948776.0      5
           XR         828135.0      7
M-SC       XY24       797985.0      6
X-SC       XY24       606538.0      4
X-LC       X40        585456.0      6
X-MC       X40        535799.0      7
           X40N       415037.0      4
X-SC       X40N       385568.0      4
H-SC       OX40N      347920.0      4
M-SC       OX40N      343684.0      5
H-SC       SR         298080.0      2
X-LC       X40N       224443.0      2
H-MC       XY24       212006.0      1
X-LC       XY24       206504.0      2
X-MC       XY25       204061.0      2
           XY24       198361.0      2
H-MC       AR         184865.0      2
M-SC       XR         176546.0      2
X-SC       X40        175106.0      2
M-LC       XR         144730.0      1
X-LC       XY25       128998.0      2
L-SC       OX40N       93015.0      1
           XR          88317.0      1
H-LC       AR          74607.0      1
M-SC       AR          74000.0      1
L-MC       XR          49260.0      1
M-MC       AR          48584.0      1
           XY25        46305.0      1
L-LC       XY25        23200.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 34.0 seconds
